In [1]:
# Paso 1: Instalación de librerías necesarias
!pip install fastapi uvicorn nest_asyncio pyngrok scikit-learn joblib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.6 MB/s eta 0:00:00


In [2]:
# Paso 2: Entrenar y guardar el modelo
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import joblib

iris = load_iris()
X, y = iris.data, iris.target
target_names = iris.target_names

model = RandomForestClassifier()
model.fit(X, y)

# Guardar modelo y nombres de clases
joblib.dump((model, target_names), "model.pkl")
print("Modelo y nombres de clase guardados como model.pkl")

Modelo y nombres de clase guardados como model.pkl


In [3]:
# Paso 3: Crear la API FastAPI con salida detallada
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
import numpy as np
import logging

logging.basicConfig(level=logging.INFO)

app = FastAPI(title="Servicio de Clasificación Iris")

# Cargar modelo y clases
model, class_names = joblib.load("model.pkl")

class InputData(BaseModel):
    features: list[float]

@app.post("/predict")
def predict(data: InputData):
    try:
        X = np.array(data.features).reshape(1, -1)
        prediction = model.predict(X)[0]
        proba = model.predict_proba(X)[0]

        result = {
            "predicted_class": int(prediction),
            "class_name": class_names[prediction],
            "probabilities": {
                class_names[i]: f"{round(p * 100, 2)}%" for i, p in enumerate(proba)
            }
        }

        logging.info(f"Entrada: {data.features} -> Predicción: {result}")
        return result
    except Exception as e:
        logging.error(f"Error en la predicción: {e}")
        raise HTTPException(status_code=500, detail="Error interno del modelo")

In [4]:
# Paso 4: Ejecutar FastAPI con ngrok y manejo de errores
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import socket
import os
import time

nest_asyncio.apply()

!ngrok config add-authtoken 2uxqXsEpzKAdvsgXYMMk6JZmRkD_2a3ZiXFm2NechpL1hGYQt

def kill_process_on_port(port):
    try:
        result = os.popen(f"lsof -i :{port}").read()
        lines = result.strip().split("\\n")
        if len(lines) > 1:
            pid = int(lines[1].split()[1])
            os.kill(pid, 9)
            print(f"Proceso en el puerto {port} eliminado.")
        else:
            print(f"ℹ️ No hay procesos corriendo en el puerto {port}.")
    except Exception as e:
        print(f"Error matando proceso: {e}")

def get_free_port(start=7860, end=8081):
    for port in range(start, end):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(("127.0.0.1", port))
                return port
            except OSError:
                continue
    raise RuntimeError("No hay puertos libres disponibles")

try:
    port = get_free_port()
    kill_process_on_port(port)

    try:
        ngrok.kill()
        print("🔌 Túneles ngrok anteriores cerrados.")
    except:
        pass

    time.sleep(2)
    public_url = ngrok.connect(port)
    print(f"Tu API está disponible en: {public_url}")

    uvicorn.run(app, host="0.0.0.0", port=port)

except Exception as e:
    print("Error al iniciar el servidor o túnel ngrok:")
    print(str(e))
    print("Cierra sesiones activas desde https://dashboard.ngrok.com/agents si es necesario.")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
ℹ️ No hay procesos corriendo en el puerto 7860.
🔌 Túneles ngrok anteriores cerrados.


INFO:     Started server process [686]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:7860 (Press CTRL+C to quit)


🚀 Tu API está disponible en: NgrokTunnel: "https://d1cc-34-44-31-207.ngrok-free.app" -> "http://localhost:7860"
INFO:     223.27.115.62:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     223.27.115.62:0 - "POST /predict HTTP/1.1" 422 Unprocessable Entity
INFO:     223.27.115.62:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     223.27.115.62:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [686]
